# W205 Project 3 
## By: Dahler Battle

### Context
We are data scientists at a game development company and are trying to build a data pipeline for interesting events in our game. Our latest mobile game has three events we're interested in tracking: `buy a sword`,`join guild`, and `win_victory_points`. Each of these events have metadata characterstics attached (i.e. sword type,  of such events (i.e., sword type, guild name, victory point color, etc)

### Key business questions
Our key business questions are the following: 
1. How many unique players play in one day, i..e, what is our Daily Active Usage (DAU)?
2. Which are the top 3 most popular 'sword types', purchased by players?
3. Which are the top 3 most popular 'guilds' joined by players?
4. Which victory point colors were obtained the most between gold, silver, and bronze? 
5. What was the average strength of each sword type?

### Key events
Each of these events have sub-events that we will track throughout the game, developed in our original API. 
1. purchase_a_sword() <player purchases swords> 
    - sword_type: Generated at random from a variety of options
    - sword_level: Generated at random from 1-100
    - timestamp
2. join_guild() <player joins a guild>
    - guild_type: Generated at random from a variety of options
    - guild_strenth: Generated at random from 1-100
    - timestamp
3. victory_points() <player obtains victory points in the game>
    - victory_point_color: Generated at random between gold, silver, and bronze
    - location: Generated at random from a variety of options
    - timestamp

### Spin up and check the cluster
Notice that we will use the following containers in our cluster below.  These each come from our `docker-compose.yml` file enclosed. Notice that one new one, Cloudera, exposes multiple ports in order to accomplish table formating functions in Hive. 
- Zookeeper
- MIDS
- Cloudera
- Presto
- Kafka
- Spark

In [4]:
!docker-compose up -d
print("\n")
!docker ps

Creating network "project3dahlerbattle_default" with the default driver
Creating project3dahlerbattle_zookeeper_1
Creating project3dahlerbattle_cloudera_1
Creating project3dahlerbattle_presto_1
Creating project3dahlerbattle_mids_1
Creating project3dahlerbattle_kafka_1
Creating project3dahlerbattle_spark_1


CONTAINER ID        IMAGE                              COMMAND                  CREATED             STATUS                  PORTS                                                              NAMES
7d16952899cd        midsw205/spark-python:0.0.6        "docker-entrypoint.s…"   1 second ago        Up Less than a second   8888/tcp                                                           project3dahlerbattle_spark_1
8076e9b8acf0        confluentinc/cp-kafka:latest       "/etc/confluent/dock…"   2 seconds ago       Up Less than a second   9092/tcp, 29092/tcp                                                project3dahlerbattle_kafka_1
24599e0be854        midsw205/base:0.1.9               

### Execute our Game API through Flask and our `game_api.py` file.
Next we will instrument our API, which tracks events `buy a sword`,`join guild`, and `win_victory_points`. This can be seen through the `game_api.py` file. Then we run this API through flask to get it up and running. 

NOTE: Please execute the following command in a seperate terminal in order to continue forward in the notebook. Leave this terminal window up. 

In [11]:
#!docker-compose exec mids env FLASK_APP=/w205/project-3-dahlerbattle/game_api.py flask run --host 0.0.0.0

### Set up to log events in Kafka
Next we will set up Kafka to take events from Spark streaming to HDFS/parquet so that they can later be queried. Unlike previous times, we do not have to explicityly set up a topic. Kafka will do this for us below. Notice that we run the kafkacat function without the -e so that we can prepare for streaming later on. 

NOTE: Please execute the following command in a seperate terminal and leave the terminal window up in order to continue forward. You should also run this command **twice** so that it will create the topic. 

In [12]:
#!docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning

### Set up data streaming through Spark and our `write_stream.py` file.
Finally, we will use data streaming to take incoming events into the game. We set up raw events to go through three different sinks for each of our game's variables. 

NOTE: Please execute the following command in a seperate terminal and leave the terminal window up in order to continue forward. 

In [13]:
#!docker-compose exec spark spark-submit /w205/project-3-dahlerbattle/write_stream.py

### Check out in Hadoop
We want to ensure that it streams are able to flow into the HDFS. The following command should show three primary directories, `sword_purchases`, `join_guild`, and `win_victory_points`. It should also include checkpoint directories for these streams as well to store metadata and other related misc. items. 

In [6]:
!docker-compose exec cloudera hadoop fs -ls /tmp/

Found 9 items
drwxrwxrwt   - root   supergroup          0 2020-08-03 17:46 /tmp/checkpoints_for_join_guild
drwxrwxrwt   - root   supergroup          0 2020-08-03 17:46 /tmp/checkpoints_for_sword_purchases
drwxrwxrwt   - root   supergroup          0 2020-08-03 17:46 /tmp/checkpoints_for_win_victory_points
drwxrwxrwt   - mapred mapred              0 2016-04-06 02:26 /tmp/hadoop-yarn
drwx-wx-wx   - hive   supergroup          0 2020-08-03 17:42 /tmp/hive
drwxr-xr-x   - root   supergroup          0 2020-08-03 17:46 /tmp/join_guild
drwxrwxrwt   - mapred hadoop              0 2016-04-06 02:28 /tmp/logs
drwxr-xr-x   - root   supergroup          0 2020-08-03 17:46 /tmp/sword_purchases
drwxr-xr-x   - root   supergroup          0 2020-08-03 17:46 /tmp/win_victory_points


### Setting up Presto

#### Setting up the tables through Hive.
Next run hive through the hadoop container. There are easier ways to arrange these tables, such as going through Apache Spark. However, this was the one that worked best for the purposes of this project. 

NOTE: Please execute the following command in a seperate terminal. The first command will open Hive and the second set of commands (3 commands in total) should be inserted into the Hive engine to create the tables. Exit out of Hive (Control-D) so that we can use the same terminal for our Presto queries. 

In [14]:
#!docker-compose exec cloudera hive

In [15]:
#create external table if not exists default.sword_purchases (raw_event string, timestamp string, accept string, host string, user_agent string, event_type string, sword_type string, sword_level string) stored as parquet location '/tmp/sword_purchases'  tblproperties ("parquet.compress"="SNAPPY");

#create external table if not exists default.join_guild (raw_event string, timestamp string, accept string, host string, user_agent string, event_type string, guild_type string, guild_strength string) stored as parquet location '/tmp/join_guild'  tblproperties ("parquet.compress"="SNAPPY");

#create external table if not exists default.win_victory_points (raw_event string, timestamp string, accept string, user_agent string, host string, event_type string, victory_point_color string, location string) stored as parquet location '/tmp/win_victory_points'  tblproperties ("parquet.compress"="SNAPPY");


#### Showing that Presto can run queries
Presto can then be opened up via its new docker container. We can show that tables have been created and that they go by a certain schema created in hive. However, notice that no data has been inserted into our system. It is time to put data into our streaming environment! 

Note: Please execute the following commands in a seperate terminal. The first command should open Presto and the rest should be inserted into the query engine directly. 

In [16]:
#!docker-compose exec presto presto --server presto:8080 --catalog hive --schema default

#insert the following SQL-based queries
#show tables;
#describe sword_purchases;
#select count(*) from win_victory_points;

#exit out of presto using control-d

## Add some data into our stream

The following pumps events into our pipeline now that the stream is fully running. 

In [7]:
#Feeding Data

!docker-compose exec mids ab -n 100 -H "Host: user1.usa.gov" http://localhost:5000/purchase_a_sword
        
!docker-compose exec mids ab -n 100 -H "Host: person2.gmail.com" http://localhost:5000/join_guild
        
!docker-compose exec mids ab -n 100 -H "Host: rando3.att.net" http://localhost:5000/win_victory_points
        
!docker-compose exec mids ab -n 100 -H "Host: computer1.gmail.com" http://localhost:5000/

This is ApacheBench, Version 2.3 <$Revision: 1706008 $>
Copyright 1996 Adam Twiss, Zeus Technology Ltd, http://www.zeustech.net/
Licensed to The Apache Software Foundation, http://www.apache.org/

Benchmarking localhost (be patient).....done


Server Software:        Werkzeug/0.14.1
Server Hostname:        localhost
Server Port:            5000

Document Path:          /purchase_a_sword
Document Length:        17 bytes

Concurrency Level:      1
Time taken for tests:   0.331 seconds
Complete requests:      100
Failed requests:        0
Total transferred:      17200 bytes
HTML transferred:       1700 bytes
Requests per second:    301.69 [#/sec] (mean)
Time per request:       3.315 [ms] (mean)
Time per request:       3.315 [ms] (mean, across all concurrent requests)
Transfer rate:          50.68 [Kbytes/sec] received

Connection Times (ms)
              min  mean[+/-sd] median   max
Connect:        0    0   0.0      0       0
Processing:     1    3   0.9      3       8
Waiting:        1 

## Query the data in Presto to answer business-specific questions

Now that events are streaming into our pipeline, we can go back to Presto to conduct some queries on business-related topics. 

NOTE: Run the Presto commands in a seperate terminal as well. Use Control-D to exit once finished. Since our data is randomly generated, the end user will most likely come up with different results for questions 2-4. I pasted my example outputs below just to show what outputs should look like. 

In [17]:
#!docker-compose exec presto presto --server presto:8080 --catalog hive --schema default

### Question 1: Which are the top 3 most popular 'sword types', purchased by players?
NOTE: Please copy these queriies directly into your presto terminal.

In [18]:
"""
SELECT   sword_type, count(sword_type) as n_swords
FROM     sword_purchases
GROUP BY sword_type
ORDER BY COUNT(*) DESC
LIMIT    3;
"""

'\nSELECT   sword_type, count(sword_type) as n_swords\nFROM     sword_purchases\nGROUP BY sword_type\nORDER BY COUNT(*) DESC\nLIMIT    3;\n'

#### Output:
The short_sword was the most sword type with 20 swords in the game, followed by walking_cane and rapier with 14 and 13 respectively. 

  sword_type  | n_swords 
--------------+----------
 short_sword  |       20 
 walking_cane |       14 
 rapier       |       13 


### Question 2: Which are the top 3 most popular 'guilds' joined by players?
Faun guilds were the most frequent with 17 guilds. This was followed by satyr guilds (12) and boar guilds (11). 

NOTE: Please copy this query directly into your presto terminal.

In [19]:
"""
SELECT   guild_type, count(guild_type) as n_guilds
FROM     join_guild
GROUP BY guild_type
ORDER BY COUNT(*) DESC
LIMIT    3;
"""

'\nSELECT   guild_type, count(guild_type) as n_guilds\nFROM     join_guild\nGROUP BY guild_type\nORDER BY COUNT(*) DESC\nLIMIT    3;\n'

#### Output:
 guild_type | n_guilds 
------------+----------
 faun       |       17 
 satyr      |       12 
 boar       |       11 


### Question 3: Which level of victory points were obtained the most between gold, silver, and bronze?
NOTE: Please copy this query directly into your presto terminal.

In [10]:
"""
SELECT   victory_point_color, count(victory_point_color) as victory_point_level
FROM     win_victory_points
GROUP BY victory_point_color 
ORDER BY COUNT(*) DESC
LIMIT    3;
"""

'\nSELECT   victory_point_color, count(victory_point_color) as victory_point_level\nFROM     win_victory_points\nGROUP BY victory_point_color \nORDER BY COUNT(*) DESC\nLIMIT    3;\n'

#### Output:
Gold and Silver were tied for the most common level of victory point with 34 rewarded each. Bronze followed with 32 total rewarded during gameplay.

 victory_point_color | victory_point_level 
---------------------+---------------------
 silver              |                  34 
 gold                |                  34 
 bronze              |                  32 

### Question 4: How many unique players played in one day, i..e, what is our Daily Active Usage (DAU)?
NOTE: Please copy this query directly into your presto terminal.

In [8]:
"""
SELECT count(distinct(host)) as all_unique_users
FROM   sword_purchases as Swords
UNION ALL
SELECT count(distinct(host)) 
FROM   join_guild as Guilds
UNION ALL
SELECT count(distinct(host)) 
FROM   win_victory_points as Victory_points;
"""

'\nSELECT count(distinct(host)) as all_unique_users\nFROM   sword_purchases as Swords\nUNION ALL\nSELECT count(distinct(host)) \nFROM   join_guild as Guilds\nUNION ALL\nSELECT count(distinct(host)) \nFROM   win_victory_points as Victory_points;\n'

#### Output:
We can see that there were three unique players in today's game, one for each category. This is an aspect of the game that I would like to improve and see if we can stream more players into the game. 

 all_unique_users 
------------------
                1 
                1 
                1 


### Question 5: What was the average strength of each sword type?
NOTE: Please copy this query directly into your presto terminal.

In [9]:
"""
SELECT sword_type, ROUND(AVG(CAST(sword_level AS INTEGER)), 1) AS Average_Sword_Strength
FROM sword_purchases
GROUP BY sword_type
ORDER BY Average_Sword_Strength DESC;
"""

'\nSELECT sword_type, ROUND(AVG(CAST(sword_level AS INTEGER)), 1) AS Average_Sword_Strength\nFROM sword_purchases\nGROUP BY sword_type\nORDER BY Average_Sword_Strength DESC;\n'

#### Output:
The lightsaber was, on average, the strongest sword in the game with an average strength of 67.9. This was followed by the long_sword and the katana with average strengths of 66.1 and 60.4 respectively. The short sword was the weakest sword, on average, in the game with an average strength of 42.5. 

  sword_type  | Average_Sword_Strength 
--------------+------------------------
 lightsaber   |                   67.9 
 long_sword   |                   66.1 
 katana       |                   60.4 
 rapier       |                   57.4 
 walking_cane |                   57.3 
 dao          |                   54.6 
 gladius      |                   54.1 
 short_sword  |                   42.5 

### Remove docker containers
Close out the streaming and program now that our queries are complete. 

In [20]:
!docker-compose down

Stopping project3dahlerbattle_spark_1 ... 
Stopping project3dahlerbattle_kafka_1 ... 
Stopping project3dahlerbattle_mids_1 ... 
Stopping project3dahlerbattle_presto_1 ... 
Stopping project3dahlerbattle_cloudera_1 ... 
Stopping project3dahlerbattle_zookeeper_1 ... 
ping project3dahlerbattle_cloudera_1 ... donee[2KRemoving project3dahlerbattle_spark_1 ... 
Removing project3dahlerbattle_kafka_1 ... 
Removing project3dahlerbattle_mids_1 ... 
Removing project3dahlerbattle_presto_1 ... 
Removing project3dahlerbattle_cloudera_1 ... 
Removing project3dahlerbattle_zookeeper_1 ... 
Removing network project3dahlerbattle_defaulte
